# Concatenation of output files

**Author:** Michael Needham, US EPA Region 7 Air and Radiation Division

**Contact:** needham.michael@epa.gov

**Description:** Auxiliary example showing basic concatenation of files.

<div class="alert alert-block alert-warning">
This notebook is provided for information purposes only as the necessary files to include for this example are too numerous to hold on github. 
</div>

In [3]:
import numpy as np
import xarray as xr
from pathlib import Path

In [19]:
from src.utils.cmaq import get_cmaq_metadata

In [18]:
case = 'DDM_2022_36US3.011'

data_dir = Path("/work/REGIONS/users/mneedham/cmaq/analysis/examples/example_data/")

files = sorted(data_dir.glob("*HR2DAY*"))
files = np.sort([x for x in files if case in x.name])

# Display information about the files
for f in files[:3]:
    print(f.name)
print("...")
for f in files[-3:]:
    print(f.name)

print(f"\nNUMBER OF FILES: {len(files)}")

DDM_2022_36US3.011.HR2DAY.2022120.nc
DDM_2022_36US3.011.HR2DAY.2022121.nc
DDM_2022_36US3.011.HR2DAY.2022122.nc
...
DDM_2022_36US3.011.HR2DAY.2022215.nc
DDM_2022_36US3.011.HR2DAY.2022216.nc
DDM_2022_36US3.011.HR2DAY.2022217.nc

NUMBER OF FILES: 98


In [6]:
# read in each file, format the metadata, and concat to a single xr dataset

dset_list = []
for file in files:
    dset_tmp, proj = get_cmaq_metadata(xr.open_dataset(file),return_proj=True)
    dset_list.append(dset_tmp)
    
dset = xr.concat(dset_list,dim='time').isel(LAY=0)

dset

<xarray.Dataset> Size: 20MB
Dimensions:  (time: 97, VAR: 2, DATE-TIME: 2, y: 148, x: 172)
Coordinates:
  * x        (x) float64 1kB 0.0 3.6e+04 7.2e+04 ... 6.12e+06 6.156e+06
  * y        (y) float64 1kB 0.0 3.6e+04 7.2e+04 ... 5.256e+06 5.292e+06
  * time     (time) datetime64[ns] 776B 2022-04-30 2022-05-01 ... 2022-08-05
Dimensions without coordinates: VAR, DATE-TIME
Data variables:
    TFLAG    (time, VAR, DATE-TIME) int32 2kB 2022120 0 2022120 ... 0 2022217 0
    O3       (time, y, x) float32 10MB 0.02718 0.02732 ... 0.02885 0.02798
    PM25     (time, y, x) float32 10MB 4.251 4.349 4.421 ... 1.784 1.882 1.94
Attributes: (12/33)
    IOAPI_VERSION:  ioapi-3.2: $Id: init3.F90 247 2023-03-22 15:59:19Z coats ...
    EXEC_ID:        ????????????????                                         ...
    FTYPE:          1
    CDATE:          2024156
    CTIME:          204606
    WDATE:          2024156
    ...             ...
    VGLVLS:         [1.     0.9975]
    GDNAM:          36US3           
    UPNAM:          hr2day          
    VAR-LIST:       O3              PM25            
    FILEDESC:       Parameters Relevant for PM Formation and Distribution    ...
    HISTORY:

In [8]:
print(f"Total size of the dataset: {dset.nbytes / 1e6} MB")

Total size of the dataset: 19.758744 MB


In [13]:
start_date = files[0].name.split(".")[-2]
end_date = files[-1].name.split(".")[-2]

print(f"{start_date=}")
print(f"{end_date=}")

start_date='2022120'
end_date='2022217'


In [20]:
fname = f"./tutorial_data/DDM_2022_36US3.011.HR2DAY.{start_date}-{end_date}.nc"
dset.to_netcdf(fname)